### Conv Numpy Implementation

In [1]:
from typing import Dict
from typing import Optional
from typing import Tuple
from typing import Union

# third party
import numpy as np

# relative
from syft.core.tensor.nn import activations
from syft.core.tensor.autodp.gamma_tensor import GammaTensor
from syft.core.tensor.autodp.phi_tensor import PhiTensor
from syft.core.tensor.nn.initializations import XavierInitialization
from syft.core.tensor.nn.utils import dp_zeros
from syft.core.tensor.nn.layers.base import Layer

/home/shubham/anaconda3/envs/syft/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_im2col_indices(x_shape, field_height, field_width, padding=1, stride=1):
    # First figure out what the size of the output should be
    N, C, H, W = x_shape
    assert (H + 2 * padding - field_height) % stride == 0
    assert (W + 2 * padding - field_height) % stride == 0
    out_height = int((H + 2 * padding - field_height) / stride + 1)
    out_width = int((W + 2 * padding - field_width) / stride + 1)

    i0 = np.repeat(np.arange(field_height), field_width)
    i0 = np.tile(i0, C)
    i1 = stride * np.repeat(np.arange(out_height), out_width)
    j0 = np.tile(np.arange(field_width), field_height * C)
    j1 = stride * np.tile(np.arange(out_width), out_height)
    i = i0.reshape(-1, 1) + i1.reshape(1, -1)
    j = j0.reshape(-1, 1) + j1.reshape(1, -1)

    k = np.repeat(np.arange(C), field_height * field_width).reshape(-1, 1)

    return (k.astype(int), i.astype(int), j.astype(int))

def im2col_indices(x, field_height, field_width, padding=1, stride=1):
    """ An implementation of im2col based on some fancy indexing """
    # Zero-pad the input
    p = padding
    x_padded = np.pad(x, ((0, 0), (0, 0), (p, p), (p, p)), mode='constant')

    k, i, j = get_im2col_indices(x.shape, field_height, field_width, padding, stride)

    cols = x_padded[:, k, i, j]
    C = x.shape[1]
    cols = cols.transpose(1, 2, 0).reshape(field_height * field_width * C, -1)
    return cols

In [3]:
from syft import DataSubjectList
from syft import lazyrepeatarray

def dp_pad(input: Union[PhiTensor, GammaTensor], width, padding_mode="constant", **kwargs):
    
    data = input.child    
    output_data = np.pad(data, width, mode=padding_mode, **kwargs)
    min_v = lazyrepeatarray(data=min(input.min_vals.data.min(), output_data.min()), shape=output_data.shape)
    max_v = lazyrepeatarray(data=min(input.max_vals.data.max(), output_data.max()), shape=output_data.shape)
    
    output_data_subjects=DataSubjectList(
        one_hot_lookup=input.data_subjects.one_hot_lookup,
        data_subjects_indexed=np.pad(input.data_subjects.data_subjects_indexed, width, mode=padding_mode, **kwargs)
    )

    if isinstance(input, PhiTensor):
        return PhiTensor(
            child=output_data,
            data_subjects=output_data_subjects,
            min_vals=min_v,
            max_vals=max_v
        )
    elif isinstance(input, GammaTensor):
        return GammaTensor(
            child=output_data,
            data_subjects=output_data_subjects,
            min_vals=min_v,
            max_vals=max_v,
        )
    else:
        raise NotImplementedError(f"Padding is not implemented for Input Type: {type(input)}")

In [4]:
N = 10
C_in = 3
H_in = 50
W_in = 50


input_shape = (N, C_in, H_in, W_in)
x = PhiTensor(child=np.random.randint(low=0, high=255, size=input_shape),
              data_subjects=np.zeros(input_shape),
              min_vals=0,
              max_vals=255
             )

# TEST Padding
# dp_pad(x, ((0, 0), (0, 0), (p, p), (p, p)), padding_mode="constant")

In [5]:
def im2col_indices_dp(x, field_height, field_width, padding=1, stride=1):
    """ An implementation of im2col based on some fancy indexing """
    # Zero-pad the input
    p = padding
    x_padded = dp_pad(x, ((0, 0), (0, 0), (p, p), (p, p)), padding_mode='constant')

    k, i, j = get_im2col_indices(x.shape, field_height, field_width, padding, stride)

    cols = x_padded[:, k, i, j]
    C = x.shape[1]
    cols = cols.transpose((1, 2, 0)).reshape((field_height * field_width * C, -1))
    return cols

In [6]:
print("numpy implementation")
print(im2col_indices(x.child, 32, 32, padding=2, stride=1).shape)
print("dp implementation")
print(im2col_indices_dp(x, 32, 32, padding=2, stride=1).shape)

numpy implementation
(3072, 5290)
dp implementation
(3072, 5290)


In [7]:

def conv_forward(X, W, b, stride=1, padding=1):
    cache = W, b, stride, padding
    n_filters, d_filter, h_filter, w_filter = W.shape
    n_x, d_x, h_x, w_x = X.shape
    h_out = (h_x - h_filter + 2 * padding) / stride + 1
    w_out = (w_x - w_filter + 2 * padding) / stride + 1

    if not h_out.is_integer() or not w_out.is_integer():
        raise Exception('Invalid output dimension!')

    h_out, w_out = int(h_out), int(w_out)
    print(h_out, w_out)

    X_col = im2col_indices(X, h_filter, w_filter, padding=padding, stride=stride)
    W_col = W.reshape(n_filters, -1)
    
    print(W_col.shape)
    print(X_col.shape)
    #return W_col,X_col
    

    out = (W_col @ X_col).T + b
    out = out.reshape(n_filters, h_out, w_out, n_x)
    out = out.transpose(3, 0, 1, 2)

    cache = (X, W, b, stride, padding, X_col)

    return out, cache

In [8]:
from syft.core.tensor.nn.utils import dp_zeros

In [9]:
from syft.core.tensor.nn.initializations import XavierInitialization

nb_filter = 32
filter_height = filter_width = 3
nb_batch, pre_nb_filter, pre_height, pre_width = x.shape
W = XavierInitialization()((nb_filter, pre_nb_filter, filter_height, filter_width))
W = PhiTensor(
    child=W, 
    data_subjects=DataSubjectList(one_hot_lookup=x.data_subjects.one_hot_lookup, data_subjects_indexed=np.zeros_like(W)),
    min_vals=W.min(),
    max_vals=W.max()
)

b = dp_zeros(shape=(nb_filter, ), data_subjects=x.data_subjects)

out, c = conv_forward(x.child, W.child, b.child, padding=2)

52 52
(32, 27)
(27, 27040)


In [10]:
out.shape

(10, 32, 52, 52)

In [11]:
def conv_forward_dp(X, W, b, stride=1, padding=1):
    cache = W, b, stride, padding
    n_filters, d_filter, h_filter, w_filter = W.shape
    n_x, d_x, h_x, w_x = X.shape
    h_out = (h_x - h_filter + 2 * padding) / stride + 1
    w_out = (w_x - w_filter + 2 * padding) / stride + 1

    if not h_out.is_integer() or not w_out.is_integer():
        raise Exception('Invalid output dimension!')

    h_out, w_out = int(h_out), int(w_out)
    
    print(h_out, w_out)

    X_col = im2col_indices_dp(X, h_filter, w_filter, padding=padding, stride=stride)
    
    W_col = W.reshape((n_filters, -1))
    
    #return W_col, X_col
    
    
    out = X_col.transpose() @ W_col.T + b
    out = out.reshape((n_filters, h_out, w_out, n_x))
    out = out.transpose((3, 0, 1, 2))

    cache = (X, W, b, stride, padding, X_col)

    return out, cache

In [12]:
out_dp, cache_dp = conv_forward_dp(x, W.child, b.child, padding=2)

52 52


In [13]:
out_dp.shape

(10, 32, 52, 52)

In [14]:
def col2im_indices(cols, x_shape, field_height=3, field_width=3, padding=1,
                   stride=1):
    """ An implementation of col2im based on fancy indexing and np.add.at """
    N, C, H, W = x_shape
    H_padded, W_padded = H + 2 * padding, W + 2 * padding
    x_padded = np.zeros((N, C, H_padded, W_padded), dtype=cols.dtype)
    k, i, j = get_im2col_indices(x_shape, field_height, field_width, padding, stride)
    cols_reshaped = cols.reshape(C * field_height * field_width, -1, N)
    cols_reshaped = cols_reshaped.transpose(2, 0, 1)
    np.add.at(x_padded, (slice(None), k, i, j), cols_reshaped)
    if padding == 0:
        return x_padded
    return x_padded[:, :, padding:-padding, padding:-padding]


def conv_backward(dout, cache):
    X, W, b, stride, padding, X_col = cache
    n_filter, d_filter, h_filter, w_filter = W.shape

    db = np.sum(dout, axis=(0, 2, 3))
    db = db.reshape(n_filter, -1)

    dout_reshaped = dout.transpose(1, 2, 3, 0).reshape(n_filter, -1)
    dW = dout_reshaped @ X_col.T
    dW = dW.reshape(W.shape)

    W_reshape = W.reshape(n_filter, -1)
    dX_col = W_reshape.T @ dout_reshaped
    print("dX_col", dX_col.shape)
    dX = col2im_indices(dX_col, X.shape, h_filter, w_filter, padding=padding, stride=stride)
    return dX, dW, db


In [15]:
dX, dW, db = conv_backward(out, c)

dX_col (27, 27040)


In [16]:
dX.shape, dW.shape, db.shape

((10, 3, 50, 50), (32, 3, 3, 3), (32, 1))

In [17]:
def dp_add_at(a, indices, b):
    data_a = a.child
    data_b = b.child
    np.add.at(data_a,indices, data_b)
    return PhiTensor(
        child=data_a,
        data_subjects=a.data_subjects,
        min_vals=data_a.min(),
        max_vals=data_a.max()
    )

In [18]:
def col2im_indices_dp(cols, x_shape, field_height=3, field_width=3, padding=1, stride=1):
    """ An implementation of col2im based on fancy indexing and np.add.at """
    N, C, H, W = x_shape
    H_padded, W_padded = H + 2 * padding, W + 2 * padding
    x_padded = dp_zeros((N, C, H_padded, W_padded), data_subjects=cols.data_subjects)
    k, i, j = get_im2col_indices(x_shape, field_height, field_width, padding, stride)
    cols_reshaped = cols.reshape((C * field_height * field_width, -1, N))
    cols_reshaped = cols_reshaped.transpose((2, 0, 1))
    x_padded = dp_add_at(x_padded, (slice(None), k, i, j), cols_reshaped)
    if padding == 0:
        return x_padded
    return x_padded[:, :, padding:-padding, padding:-padding]

In [19]:
def conv_backward_dp(dout, cache):
    X, W, b, stride, padding, X_col = cache
    n_filter, d_filter, h_filter, w_filter = W.shape

    db = dout.sum(axis=(0, 2, 3))
    db = db.reshape((n_filter, -1))

    dout_reshaped = dout.transpose((1, 2, 3, 0)).reshape((n_filter, -1))
    
    dW = dout_reshaped @ X_col.T
    dW = dW.reshape(W.shape)

    W_reshape = W.reshape(n_filter, -1)
    dX_col = dout_reshaped.transpose() @ W_reshape
    dX = col2im_indices_dp(dX_col, X.shape, h_filter, w_filter, padding=padding, stride=stride)

    return dX, dW, db

In [20]:
dX_dp, dW_dp, db_dp = conv_backward_dp(out_dp, cache_dp)

In [21]:
dX_dp.shape, dW_dp.shape, db_dp.shape

((10, 3, 50, 50), (32, 3, 3, 3), (32, 1))

### Conv PhiTensor Implementation

In [165]:
# stdlib
from typing import Dict
from typing import Optional
from typing import Tuple
from typing import Union

# third party
import numpy as np

# relative
from syft.core.tensor.nn import activations
from syft.core.tensor.autodp.gamma_tensor import GammaTensor
from syft.core.tensor.autodp.phi_tensor import PhiTensor
from syft.core.tensor.nn.initializations import XavierInitialization
from syft.core.tensor.nn.utils import dp_zeros
from syft.core.tensor.nn.layers.base import Layer


class Convolution(Layer):
    """
    If this is the first layer in a model, provide the keyword argument `input_shape`
    (tuple of integers, does NOT include the sample axis, N.),
    e.g. `input_shape=(3, 128, 128)` for 128x128 RGB pictures.
    """
    __name__ = "ConvPointer"
    __module__ = "syft.core.tensor.nn.layers.convolution"
    __attr_allowlist__ = [
        "nb_filter",
        "filter_size",
        "input_shape",
        "stride",
        "W",
        "b",
        "dW",
        "db",
        "out_shape",
        "last_output",
        "last_input"
    ]

    def __init__(self, nb_filter, filter_size, input_shape: Optional[Tuple]=None, stride: int=1, padding: int=0, activation: Optional[activations.Activation]=None):
        self.nb_filter = nb_filter
        self.filter_size = filter_size
        self.input_shape = input_shape
        self.stride = stride
        self.padding = padding

        self.W, self.dW = None, None
        self.b, self.db = None, None
        self.out_shape = None
        self.last_output = None
        self.last_input = None

        self.init = XavierInitialization()
        self.activation = activations.get(activation)

    def connect_to(self, prev_layer: Optional[Layer]=None):
        if prev_layer is None:
            assert self.input_shape is not None
            input_shape = self.input_shape
        else:
            input_shape = prev_layer.out_shape

        # input_shape: (batch size, num input feature maps, image height, image width)
        assert len(input_shape) == 4

        nb_batch, pre_nb_filter, pre_height, pre_width = input_shape
        if isinstance(self.filter_size, tuple):
            filter_height, filter_width = self.filter_size
        elif isinstance(self.filter_size, int):
            filter_height = filter_width = self.filter_size
        else:
            raise NotImplementedError

        height = (pre_height - filter_height + 2 * self.padding) // self.stride + 1
        width = (pre_width - filter_width +  2 * self.padding) // self.stride + 1

        # output shape
        self.out_shape = (nb_batch, self.nb_filter, height, width)

        # filters
        self.W = self.init((self.nb_filter, pre_nb_filter, filter_height, filter_width))
        self.b = np.zeros((self.nb_filter,))
        
        
    def forward(self, input, *args, **kwargs):
        self.last_input = input
        
        n_filters, d_filter, h_filter, w_filter = self.W.shape
        n_x, d_x, h_x, w_x = input.shape
        
        _, _, h_out, w_out, = self.out_shape

        self.X_col = im2col_indices_dp(input, h_filter, w_filter, padding=self.padding, stride=self.stride)

        W_col = self.W.reshape((n_filters, -1))

        out = self.X_col.transpose() @ W_col.T + self.b
        out = out.reshape((n_filters, h_out, w_out, n_x))
        out = out.transpose((3, 0, 1, 2))
        
        self.last_output = self.activation.forward(out) if self.activation is not None else out
        return out
    
    def backward(self, pre_grads, *args, **kwargs):
        n_filter, d_filter, h_filter, w_filter = self.W.shape
        
        pre_grads = (pre_grads * self.activation.derivative(pre_grads)) if self.activation is not None else pre_grad
        db = pre_grads.sum(axis=(0, 2, 3))
        self.db = db.reshape((n_filter, -1))

        pre_grads_reshaped = pre_grads.transpose((1, 2, 3, 0)).reshape((n_filter, -1))

        dW = pre_grads_reshaped @ self.X_col.T
        self.dW = dW.reshape(W.shape)

        W_reshape = self.W.reshape(n_filter, -1)
        dX_col = pre_grads_reshaped.transpose() @ W_reshape
        dX = col2im_indices_dp(dX_col, self.input_shape, h_filter, w_filter, padding=self.padding, stride=self.stride)
        return dX


#     def forward(self, input: Union[PhiTensor, GammaTensor], *args: Tuple, **kwargs: Dict):
        
        
        
        

        

#         # TODO: This could fail if the DP Tensor has < 4 dimensions

#         # shape
#         nb_batch, input_depth, old_img_h, old_img_w = input.shape
#         if isinstance(self.filter_size, tuple):
#             filter_height, filter_width = self.filter_size
#         elif isinstance(self.filter_size, int):
#             filter_height = filter_width = self.filter_size
#         else:
#             raise NotImplementedError

#         new_img_h, new_img_w = self.out_shape[2:]

#         # init
#         outputs = dp_zeros((nb_batch, self.nb_filter, new_img_h, new_img_w), input.data_subjects)

#         # convolution operation
#         for x in np.arange(nb_batch):
#             for y in np.arange(self.nb_filter):
#                 for h in np.arange(new_img_h):
#                     for w in np.arange(new_img_w):
#                         h_shift, w_shift = h * self.stride, w * self.stride
#                         # patch: (input_depth, filter_h, filter_w)
#                         patch = input[x, :, h_shift: h_shift + filter_height, w_shift: w_shift + filter_width]
#                         outputs[x, y, h, w] = (patch * self.W[y]).sum() + self.b[y]

#         # nonlinear activation
#         # self.last_output: (nb_batch, output_depth, image height, image width)

#         # TODO: Min/max vals are direct function of private data- fix this when we have time

#         self.last_output = self.activation.forward(outputs) if self.activation is not None else outputs

#         return self.last_output

#     def backward(self, pre_grad, *args, **kwargs):

#         # shape
#         assert pre_grad.shape == self.last_output.shape
#         nb_batch, input_depth, old_img_h, old_img_w = self.last_input.shape
#         new_img_h, new_img_w = self.out_shape[2:]

#         if isinstance(self.filter_size, tuple):
#             filter_height, filter_width = self.filter_size
#         elif isinstance(self.filter_size, int):
#             filter_height = filter_width = self.filter_size
#         else:
#             raise NotImplementedError

#         #         filter_h, filter_w = self.filter_size
#         old_img_h, old_img_w = self.last_input.shape[-2:]

#         # gradients
#         # TODO: Decide if dW and db needs to be DP Tensors or can they live as numpy arrays
#         self.dW = np.zeros((self.W.shape))
#         self.db = np.zeros((self.b.shape))
#         delta = (pre_grad * self.activation.derivative()) if self.activation is not None else pre_grad

#         # dW
#         for r in np.arange(self.nb_filter):
#             for t in np.arange(input_depth):
#                 for h in np.arange(filter_height):
#                     for w in np.arange(filter_width):
#                         input_window = self.last_input[:, t,
#                                        h:old_img_h - filter_height + h + 1:self.stride,
#                                        w:old_img_w - filter_width + w + 1:self.stride]
#                         delta_window = delta[:, r]
#                         self.dW[r, t, h, w] = ((input_window * delta_window).sum() * (1/nb_batch)).child
#         # db
#         for r in np.arange(self.nb_filter):
#             self.db[r] = (delta[:, r].sum() * (1/nb_batch)).child

#         # dX
#         if not self.first_layer:
#             layer_grads = self.last_input.zeros_like()
#             for b in np.arange(nb_batch):
#                 for r in np.arange(self.nb_filter):
#                     for t in np.arange(input_depth):
#                         for h in np.arange(new_img_h):
#                             for w in np.arange(new_img_w):
#                                 h_shift, w_shift = h * self.stride, w * self.stride
#                                 temp = layer_grads[b, t, h_shift:h_shift + filter_height, w_shift:w_shift + filter_width]
#                                 layer_grads[b, t, h_shift:h_shift + filter_height, w_shift:w_shift + filter_width] = temp+ (delta[b, r, h, w] * self.W[r, t])

#             return layer_grads


    @property
    def params(self):
        return self.W, self.b

    @property
    def grads(self):
        return self.dW, self.db



In [1]:
from syft.core.tensor.nn import Model, leaky_ReLU, Convolution

/home/shubham/anaconda3/envs/syft/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
net = Model()

In [28]:
net.add(Convolution(32, (3, 3), input_shape=(10, 3, 50, 50), padding=2, activation=leaky_ReLU()))

In [29]:
net.compile()

In [30]:
conv = net.layers[0]

In [31]:
x.shape

(10, 3, 50, 50)

In [32]:
conv.W.shape

(32, 3, 3, 3)

In [33]:
out = conv.forward(x)

In [34]:
conv.backward(out).shape

(10, 3, 50, 50)

In [35]:
conv.b.shape, conv.W.shape, conv.db.shape, conv.dW.shape

((32,), (32, 3, 3, 3), (32, 1), (32, 3, 3, 3))

In [1]:
from syft import PhiTensor
import numpy as np

/home/shubham/anaconda3/envs/syft/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
N = 10
C_in = 3
H_in = 50
W_in = 50


input_shape = (N, C_in, H_in, W_in)
x = PhiTensor(child=np.random.randint(low=0, high=255, size=input_shape),
              data_subjects=np.zeros(input_shape),
              min_vals=0,
              max_vals=255
             )

# TEST Padding
# dp_pad(x, ((0, 0), (0, 0), (p, p), (p, p)), padding_mode="constant")

In [3]:
from syft.core.tensor.nn import Convolution, Model, leaky_ReLU, MaxPool

In [4]:
net = Model()
net.add(Convolution(32, (3, 3), input_shape=(10, 3, 50, 50), padding=2, activation=leaky_ReLU()))
net.add(MaxPool(pool_size=2, stride=2))
net.compile()
conv = net.layers[0]
out = conv.forward(x)
print("Forward pass", out.shape)
conv.backward(out).shape

Forward pass (10, 32, 52, 52)


(10, 3, 50, 50)

In [6]:
X_col, max_idx, h_out, w_out, n, d = net.layers[1].forward(out)

In [11]:
X_col.shape

(4, 216320)

In [7]:
X_col.data_subjects.data_subjects_indexed.shape

(320, 4, 676)

In [13]:
(4, 320 * 676)

(4, 216320)

In [10]:
max_idx.size

216320

In [9]:
X_col[max_idx, range(max_idx.size)]

IndexError: index 4 is out of bounds for axis 1 with size 4

In [ ]:
(5, 5) 

(3, 5, 5)







In [ ]:
dsl_index = [0,:]

In [ ]:
data = np.random.rand(4, 6)

dsl_index = np.ones(3, 4, 6)




In [15]:
outputs = X_col.child[max_idx, range(max_idx.size)]

In [17]:
outputs.reshape((h_out, w_out, n, d)).shape

(26, 26, 10, 32)

In [19]:
pre_grad = outputs.reshape((h_out, w_out, n, d)).transpose((2, 3, 0, 1))
pre_grad.shape

(10, 32, 26, 26)

In [21]:
pre_grad_dp = PhiTensor(pre_grad, data_subjects=out.data_subjects, min_vals=pre_grad.min(), max_vals=pre_grad.max())

In [22]:
net.layers[1].backward(pre_grad_dp)

TypeError: __init__() missing 3 required positional arguments: 'data_subjects', 'min_vals', and 'max_vals'

In [31]:
dout_col = pre_grad.transpose((2, 3, 0, 1)).ravel()

In [34]:
dout_col.size

216320

In [37]:
X_col.zeros_like().child[max_idx, range(dout_col.size)] = dout_col

In [ ]:
X_col